이 파일은 item x item 상호작용 matrix를 cluster x item 상호작용 matrix로 계산 및 변환하는 파일입니다!!

In [1]:
import pandas as pd
import numpy as np
import openpyxl

from collections import Counter
from itertools import permutations
from tqdm import tqdm

In [2]:
data_path = '/opt/ml/input/data/raw_codishop/view/codi/codi_item_id.xlsx'
# preprocessed_path = '/opt/ml/input/data/asset_codishop/view/item/item.xlsx'
preprocessed_path = './temp_color_midclass.xlsx'

item_df = pd.read_excel(data_path, engine='openpyxl')
preprocessed_df = pd.read_excel(preprocessed_path, engine='openpyxl')

In [3]:
drop_indexes = list()
for i in range(len(item_df)):
    if item_df.iloc[i]["item_id"] not in preprocessed_df["id"].unique() :
        drop_indexes.append(i)

item_df = item_df.drop(drop_indexes)

item x item 상호작용 행렬을 생성하는 파일

In [4]:
items_in_codi = item_df.groupby('id')['item_id'].apply(list)

In [5]:
# 코디 내에서 가능한 아이템들의 조합을 계산하는 코드
possible_combinations = list()
items_in_codi = item_df.groupby('id')['item_id'].apply(list)

for val in tqdm(items_in_codi):
    val = sorted(val)
    comb_res = list(permutations(val, 2))
    possible_combinations.extend(comb_res)

100%|██████████| 400/400 [00:00<00:00, 322081.32it/s]


In [6]:
pivot_list = list()
for item1, item2 in possible_combinations:
    cluster_id = preprocessed_df[preprocessed_df['id'] == item1]['cluster_id'].values
    pivot_list.append((cluster_id[0], item2))

In [7]:
count_combination = list(Counter(pivot_list).most_common())

In [8]:
result = list()
for (cluster_id, item_id), cnt in count_combination:
    result.append((cluster_id, item_id, cnt))

In [9]:
IIM = pd.DataFrame(result, columns=["id", "item", "count"]).pivot_table(index="id", columns="item", values="count").fillna(0)
IIM = IIM.astype(int)
# TODO : 데이터 경로 설정
IIM.to_csv("/opt/ml/input/data/asset_codishop/view/item/itemInteractionMatrix_withColor.csv", index=True)

In [10]:
IIM

item,102620,102621,104238,104239,165980,166001,181907,186610,212880,218237,...,2083412,2116419,2116421,2116440,2138587,2139333,2157202,2201846,2224536,2284131
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
ones = 0
zeros = 0
for i in range(len(IIM)) :
    oz_cnt = IIM.iloc[i].value_counts()
    zeros += oz_cnt[0]
    ones += sum(oz_cnt[1:])
print(f"Sparsity of this Data is : {zeros/(ones+zeros):.4}")

Sparsity of this Data is : 0.9943
